In [0]:
def f(x,y):
          num = x*y
          return num
  
def g(x,y):
          num = (x+2)*(y+3.5)-200
          return num



In [91]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    #res = x**2 + y**2 + z**2
    res = f(x,y)+z*g(x,y)
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 20
    child_num = 15

    space = OrderedDict((
        ('x', (-15., 15.)),
        ('y', (-15., 15.)),
        ('z', (0., 2.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 14.359776515649976, -13.229854106489455, 0.20901739384419216: -265.0522476849818
Eval -14.936703131637334, 13.902043592491367, 0.27543303443083134: -324.74418661360573
Eval 5.689564384558452, -10.539092124974733, 1.4127939847667634: -418.9927201756322
Eval -6.314675841591878, 5.853841456794296, 1.8193846880944033: -474.2702202429866
Eval -13.099385825162832, -7.208494002399566, 0.11328208284781205: 76.43334533326069
Eval -5.510486248728753, 0.5228725844707576, 0.9964615989466885: -216.24587071911654
Eval 4.037508381539798, -14.094026032354522, 0.6354543299626356: -224.640239675744
Eval -7.088377226645549, 7.423494128788285, 0.39065050830356585: -152.46410041303835
Eval -8.227449772522053, 7.413731069849156, 0.9463050802343529: -314.57246835793705
Eval -2.1231469442560957, -6.3553153648000045, 1.6527982156219345: -316.48521226243986
Eval -8.530953136627879, -5.9563241565725775, 1.1902835208185414: -168.1488894709573
Eval 8.721611051012964, -0.35244785233417986, 0.6752016799242295: 